In [1]:
# # !pip install diffusers==0.11.1
# !pip install diffusers["torch"] transformers
# !pip install accelerate
# !pip install ipywidgets
# !pip install safetensors
# !git clone https://github.com/huggingface/diffusers.git
!pip install transformers scipy ftfy accelerate

In [2]:
!cd ./diffusers

# assume you have downloaded xxx.safetensors, it will out save_dir in diffusers format.
!python diffusers/scripts/convert_original_stable_diffusion_to_diffusers.py  --dump_path save_dir --from_safetensors

# # assume you have downloaded xxx.ckpt, it will out save_dir in diffusers format.
# !python ./scripts/convert_original_stable_diffusion_to_diffusers.py --checkpoint_path xxx.ckpt  --dump_path save_dir

usage: convert_original_stable_diffusion_to_diffusers.py [-h]
                                                         --checkpoint_path
                                                         CHECKPOINT_PATH
                                                         [--original_config_file ORIGINAL_CONFIG_FILE]
                                                         [--num_in_channels NUM_IN_CHANNELS]
                                                         [--scheduler_type SCHEDULER_TYPE]
                                                         [--pipeline_type PIPELINE_TYPE]
                                                         [--image_size IMAGE_SIZE]
                                                         [--prediction_type PREDICTION_TYPE]
                                                         [--extract_ema]
                                                         [--upcast_attention]
                                                         [--from_safetensors]
         

In [3]:
from diffusers import DDPMPipeline
image_pipe = DDPMPipeline.from_pretrained("models/ddpm-celebahg-256")
image_pipe.to("cuda")

DDPMPipeline {
  "_class_name": "DDPMPipeline",
  "_diffusers_version": "0.18.2",
  "scheduler": [
    "diffusers",
    "DDPMScheduler"
  ],
  "unet": [
    "diffusers",
    "UNet2DModel"
  ]
}

In [4]:
# images = image_pipe().images
images, images[0]

NameError: name 'images' is not defined

In [ ]:
image_pipe

In [7]:
from diffusers import UNet2DModel

repo_id = "models/ddpm-church-256"
model = UNet2DModel.from_pretrained(repo_id)
model

UNet2DModel(
  (conv_in): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=128, out_features=512, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=512, out_features=512, bias=True)
  )
  (down_blocks): ModuleList(
    (0-1): 2 x DownBlock2D(
      (resnets): ModuleList(
        (0-1): 2 x ResnetBlock2D(
          (norm1): GroupNorm(32, 128, eps=1e-06, affine=True)
          (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (time_emb_proj): Linear(in_features=512, out_features=128, bias=True)
          (norm2): GroupNorm(32, 128, eps=1e-06, affine=True)
          (dropout): Dropout(p=0.0, inplace=False)
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (nonlinearity): SiLU()
        )
      )
      (downsamplers): ModuleList(
        (0): Downsample2D(
          (conv): Con

In [10]:
from diffusers import DiffusionPipeline

model_id = "runwayml/stable-diffusion-v1-5"
pipeline = DiffusionPipeline.from_pretrained(model_id)

ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/runwayml/stable-diffusion-v1-5 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fbb936c6590>: Failed to establish a new connection: [Errno 101] Network is unreachable'))"), '(Request ID: 64343d74-b160-49a0-ac52-ba17962c7353)')

In [ ]:
model.config

sample_size：定义输入样本的 和 维度。heightwidth
in_channels：定义输入样本的输入通道数。
down_block_types和 ：定义用于创建 UNet 体系结构的下采样和上采样块的类型，如本笔记本开头的图所示。up_block_types
block_out_channels：定义下采样模块的输出通道数，也以相反的顺序用于上采样块的输入通道数。
layers_per_block：定义每个 UNet 块中存在多少个 ResNet 块。

使用save_pretrained()方法将模型权重和模型配置保存在提供的文件夹中:
model_random.save_pretrained("my_model")

In [ ]:
#   创建一个随机初始化模型，模型配置和之前的相同
model_random = UNet2DModel(**model.config)
model_random

use a random gaussian sample in the shape of an image (batch_size  ×  in_channels  ×  sample_size  ×  sample_size). We have a batch axis because a model can receive multiple random noises, a channel axis because each one consists of multiple channels (such as red-green-blue), and finally sample_size corresponds to the height and width.

In [ ]:
import torch

torch.manual_seed(0)

noisy_sample = torch.randn(
    1, model.config.in_channels, model.config.sample_size, model.config.sample_size
)
print("noisy_sample:")
noisy_sample.shape

with torch.no_grad():
    noisy_residual = model(sample=noisy_sample, timestep=2).sample
print("noisy_residual:")
noisy_residual.shape



In [ ]:
from diffusers import DDPMScheduler

scheduler = DDPMScheduler.from_config(repo_id)
scheduler.config

In [ ]:
less_noisy_sample = scheduler.step(
    model_output=noisy_residual, timestep=2, sample=noisy_sample
).prev_sample
less_noisy_sample.shape

In [ ]:
import tqdm
import PIL.Image
import numpy as np

def display_sample(sample, i):
    image_processed = sample.cpu().permute(0, 2, 3, 1)
    image_processed = (image_processed + 1.0) * 127.5
    image_processed = image_processed.numpy().astype(np.uint8)

    image_pil = PIL.Image.fromarray(image_processed[0])
    display(f"Image at step {i}")
    display(image_pil)
model.to("cuda")
noisy_sample = noisy_sample.to("cuda")
sample = noisy_sample

for i, t in enumerate(tqdm.tqdm(scheduler.timesteps)):
    # 1. predict noise residual
    with torch.no_grad():
        residual = model(sample, t).sample

    # 2. compute less noisy image and set x_t -> x_t-1
    sample = scheduler.step(residual, t, sample).prev_sample

    # 3. optionally look at image
    if (i + 1) % 50 == 0:
        display_sample(sample, i + 1)

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)